# Regression Model to determine influence of artist's gender on the topics generated by BERTopic

In [ ]:
# from google.colab import drive
# import os

# gdrive_path='/content/gdrive/MyDrive/Bertopic/shared_work/'

# # This will mount your google drive under 'MyDrive'
# drive.mount('/content/gdrive', force_remount=True)
# # In order to access the files in this notebook we have to navigate to the correct folder
# os.chdir(gdrive_path)
# dataset_path = ''
# # Check manually if all files are present
# print(sorted(os.listdir()))

# # To run from the common drive:
# # dataset_path = '/content/drive/MyDrive/Praktikum - NLP Applications/Models/bertopic_concatenated_chunks_stratified'
# # drive.mount('/content/drive', force_remount=True)

Mounted at /content/gdrive
['BERTopic.ipynb', 'BERTopic_female.ipynb', 'Data-PreProcessing.ipynb', 'Dataset_Statistics.ipynb', 'Regression.ipynb', 'chunk_5.csv', 'chunk_6.csv', 'concatenated_chunks.csv', 'genius_with_artists_first_chunk.csv', 'glove.6B.100d.txt', 'glove.6B.100d.txt.word2vec', 'glove.6B.200d.txt', 'glove.6B.300d.txt', 'glove.6B.50d.txt', 'glove.6B.zip', 'model', 'model_new', 'my_model', 'mymodel', 'topics.pkl', 'topics_for_bias_analysis.csv', 'updated_artists_with_gender_lyrics.csv', 'wasabi_artist_new_dartists.csv', 'wasabi_english_songs.csv', 'wasabi_new_dartists_balanced.csv', 'wasabi_new_songs_english_balanced_artists.csv']


In [ ]:
import pandas as pd
# import numpy as np
# from sklearn.feature_extraction.text import CountVectorizer
# from bertopic import BERTopic
# from umap import UMAP

# set path of the file
csv_file_path = 'topics_for_bias_analysis.csv'
df = pd.read_csv(dataset_path + csv_file_path)

In [ ]:
df.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,title,genre,artist,year,views,features,lyrics,language,gender,cleaned_lyrics,has_lyrics,topic,topic_prob,topic_label
0,0,0,0,Killa Cam,rap,Cam'ron,2004,173166,"{""Cam\\'ron"",""Opera Steve""}","[Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki...",en,Male,"\nKilla Cam, Killa Cam, Cam\nKilla Cam, Killa ...",True,7,0.609817,mi_dem_yuh
1,1,1,1,Down and Out,rap,Cam'ron,2004,144404,"{""Cam\\'ron"",""Kanye West"",""Syleena Johnson""}",[Produced by Kanye West and Brian Miller]\n\n[...,en,Male,"\n\n\nUgh, Killa!\nBaby!\nKanye, this that 197...",True,0,0.675691,nigga_niggas_bitch
2,2,2,2,Family Ties,rap,Cam'ron,2004,41960,"{""Cam\\'ron"",""Lady Wray""}","[Verse 1: Cam'ron]\nKilla, Dipset\nMan I spit ...",en,Male,"\nKilla, Dipset\nMan I spit that pimp talk, yo...",True,434,0.652398,bicyclette_ma bicyclette_bicyclette ma
3,3,3,3,Rockin and Rollin,rap,Cam'ron,1998,6399,"{""Cam\\'ron""}",[Verse 1]\nAy yo you wonder who I are\nI guzzl...,en,Male,\nAy yo you wonder who I are\nI guzzle up at t...,True,434,0.742198,bicyclette_ma bicyclette_bicyclette ma
4,4,4,4,Lord You Know,rap,Cam'ron,2004,11882,"{""Cam\\'ron"",""Juelz Santana"",Jaheim}","[Chorus: Jaheim]\nNow Lord you know, just how ...",en,Male,"\nNow Lord you know, just how hard I try\nTo l...",True,398,0.737625,dog status_movin rank_status im


In [ ]:
dataset = df[['genre', 'year', 'gender', 'topic', 'topic_prob']]

In [ ]:
dataset.describe()

,year,topic,topic_prob
count,537553.000000,537553.000000,537553.000000
mean,2002.695682,165.225821,0.684348
std,23.488690,156.126712,0.093572
min,1.000000,-1.000000,0.050249
25%,1996.000000,28.000000,0.630626
50%,2008.000000,110.000000,0.693565
75%,2015.000000,269.000000,0.746582
max,2023.000000,541.000000,0.995553


In [ ]:
# Converting year to decade
dataset['year'].unique()

array([2004, 1998, 2002, 2009, 2000, 2006, 2010, 2011, 2003, 2014, 2007,
       2005, 1999, 2013, 2015, 2008, 2001, 2020, 2012, 2016, 2019, 1995,
       1994, 1997, 1993, 2017, 1992, 1996, 1988, 1990, 1991, 1960, 2018,
       1989, 1965, 1975, 1962, 1974, 1964, 1963, 1976, 1966, 1967, 1985,
       1979, 1986, 1961, 1981, 1980, 1978, 1969, 1970, 1971, 1968, 1973,
       1983, 1987, 1972, 1977, 1982, 2022, 1984, 2021, 1956,    1, 1950,
       1953, 1951, 1949, 1958, 1955, 1957, 1946, 1959, 1954, 1952, 1945,
       1948, 1947, 1939, 1871, 1876, 1865, 1415, 2023, 1770, 1944, 1855,
       1936,   15, 1814, 1759, 1785, 1788, 1789, 1791, 1782, 1929, 1938,
       1926, 1931, 1776, 1801, 1802, 1803, 1804, 1805, 1806, 1807, 1808,
       1940, 1941, 1932, 1933, 1943, 1942,    2,  699, 1861, 1937, 1935,
       1934, 1862, 1918, 1919, 1917, 1930, 1517, 1790, 1787, 1928, 1927,
       1895, 1907, 1911, 1925, 1906, 1904, 1923,   79, 1864, 1847, 1921,
       1741, 1920, 1905, 1870, 1924, 1922, 1739, 17

In [ ]:
dataset['year'].max()

2023

In [ ]:
# Number of songs with year < 1950 - a few of these are from the 18th and 19th centuries but there are also some mistagged songs

print(len(dataset[dataset['year'] < 1950])) # => 2856, only 0.5% of the data so these are ignored
dataset = dataset[dataset['year'] >= 1950]

def get_decade(year):
  if year >= 2000 and year < 2010:
    decade = '2000s'
  else:
    decade = str(year//100) + str(year%100)[0] + '0s'
  return decade

dataset['decade'] = dataset['year'].apply(get_decade)

2856


In [ ]:
# Number of outliers

len(df[df['topic'] == -1])

8598

In [ ]:
dataset = dataset[dataset['topic'] != -1]
dataset = dataset[dataset['genre'] != 'misc']

In [ ]:
top10topics = dataset['topic'].value_counts().nlargest(10).to_dict()

In [ ]:
dataset['decade'] = dataset['year'].apply(get_decade)

In [ ]:
mask = dataset['topic'].isin(top10topics)
top10 = dataset[mask]

In [ ]:
top10_X = top10[['genre', 'decade', 'gender']]
top10_y = top10['topic']

In [ ]:
top10

,genre,year,gender,topic,topic_prob,decade
1,rap,2004,Male,0,0.675691,2000s
5,rap,2002,Male,0,0.776709,2000s
6,rap,2004,Male,0,0.778837,2000s
7,rap,2004,Male,0,0.740868,2000s
11,rap,2006,Male,0,0.706905,2000s
...,...,...,...,...,...,...
537546,pop,2020,Male,1,0.795815,2020s
537547,pop,2020,Male,1,0.770242,2020s
537548,pop,2020,Male,1,0.745915,2020s
537549,pop,2022,Female,3,0.539879,2020s


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(top10_X, top10_y, test_size=0.2, random_state=42)

In [ ]:
len(top10_y.unique())

10

The feature importances you've obtained from the RandomForestClassifier can give you insight into which features your model finds most important when making predictions. The values are a measure of the (normalized) total reduction of the criterion brought by that feature, also known as "Gini importance". if dont wan to use random forest then dont run the model here

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# le = LabelEncoder()
cat = OneHotEncoder()
# cat_dec = OneHotEncoder()
preprocessor = ColumnTransformer(transformers=[
    ('cat', cat, ['genre', 'decade', 'gender'])
])


# Create the Random Forest model pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Define the parameter grid for RandomForestClassifier
param_grid = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__max_depth': [None, 10, 20, 30],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4],
}

# Create the Grid Search model
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)

# Fit the model
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Evaluate the best model
y_pred = best_model.predict(X_test)
print("Accuracy of the best model:", accuracy_score(y_test, y_pred))

# Get feature importances from the best model
best_feature_importances = best_model.named_steps['classifier'].feature_importances_
print("Best Model Feature Importances:", best_feature_importances)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Accuracy of the best model: 0.47546184944475717
Best Model Feature Importances: [0.02637388 0.22531513 0.48226886 0.04966822 0.04158685 0.00261849
 0.00909852 0.01068829 0.01037405 0.00491283 0.00532755 0.01128898
 0.00490475 0.06458161 0.05099198]


In [27]:
# Get the feature names after one-hot encoding from the fitted pipeline
feature_names = best_model.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out()

# Create a dictionary mapping feature names to their importances
feature_importance_dict = dict(zip(feature_names, best_feature_importances))

# Print the dictionary sorted by importance
for feature, importance in sorted(feature_importance_dict.items(), key=lambda item: item[1], reverse=True):
    print(f"{feature}: {importance}")


genre_rap: 0.4822688613573007
genre_pop: 0.22531513171986564
gender_Female: 0.06458160860584904
gender_Male: 0.05099198397353409
genre_rb: 0.049668216614876484
genre_rock: 0.041586845867163935
genre_country: 0.026373877058553975
decade_2010s: 0.011288982707093336
decade_1970s: 0.010688287965213046
decade_1980s: 0.010374049915378714
decade_1960s: 0.009098523445421098
decade_2000s: 0.005327554036109543
decade_1990s: 0.004912833195236801
decade_2020s: 0.0049047489054834005
decade_1950s: 0.0026184946329202716


Analysis:
Genre Importance:

The feature genre_rap has the highest importance score (approximately 0.4823), meaning that the genre being rap is the most influential in predicting the topic, according to the model.
Following that, genre_pop has the next highest score (approximately 0.2253), suggesting that pop is also a significant predictor, but less so than rap.
Other genres like R&B (genre_rb), rock (genre_rock), and country (genre_country) have lower importance scores, indicating they have some, but less, influence on the topic compared to rap and pop.
Gender Importance:

The gender of the artist (gender_Female and gender_Male) has lower importance scores than the top genres but still carries weight in the model’s predictions, with gender_Female having a slightly higher score than gender_Male.
Decade Importance:

The decades have even lower importance scores, with the most recent decades (decade_2010s and decade_2020s) and the earliest (decade_1950s) having the least influence on the topic according to the model.
This suggests that the decade of the art is less predictive of the topic than the genre or gender.

In [45]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(top10_X, top10_y, test_size=0.2, random_state=42)

# le = LabelEncoder()
cat = OneHotEncoder()
# cat_dec = OneHotEncoder()

preprocessor = ColumnTransformer(transformers=[
    ('cat', cat, ['genre', 'decade', 'gender'])
])
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [46]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(multi_class='multinomial', solver='saga',penalty= 'l1', max_iter=1000)

In [ ]:
# note: running the grid search here is optional cuz after gridsearch the accuracy did not improve that much

#from sklearn.model_selection import GridSearchCV

# # Define a set of parameters for tuning
# param_grid = {
#     'C': [0.001, 0.01, 0.1, 1, 10, 100],  # Regularization strength
#     'penalty': ['l1', 'l2'],  # Type of penalty
#     'solver': ['newton-cg', 'lbfgs', 'liblinear', 'saga'],  # Optimization algorithms
# }
# # Create the Logistic Regression model
# model = LogisticRegression(multi_class='multinomial', max_iter=1000, solver='lbfgs')

# # Create GridSearchCV with the model and parameter grid
# grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')

# # Fit the grid search to the data
# grid_search.fit(X_train, y_train)

# # The best hyperparameters from GridSearchCV
# print("Best Parameters: ", grid_search.best_params_)

# # Use the best model
# best_model = grid_search.best_estimator_
# best_model.fit(X_train, y_train)
# # Make predictions with the best model
# y_hat = best_model.predict(X_test)
# accuracy_score(y_hat, y_test)

In [47]:
model.fit(X_train, y_train)
y_hat = model.predict(X_test)

In [39]:
from sklearn.metrics import accuracy_score

accuracy_score(y_hat, y_test)

0.4733125223888235

In [49]:
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

# Get the coefficients
coefficients = log_reg.coef_[0]  # Assuming binary classification

# Retrieve feature names from OneHotEncoder
feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(['genre', 'decade', 'gender'])

# Match coefficients with feature names
feature_importance_dict = dict(zip(feature_names, coefficients))

# Print sorted by absolute importance
sorted_importances = sorted(feature_importance_dict.items(), key=lambda item: abs(item[1]), reverse=True)
for feature, importance in sorted_importances:
    print(f"{feature}: {importance}")

genre_rap: 3.080138224392389
genre_country: -1.7997420978630612
genre_rock: -1.4428906994304644
decade_1960s: -1.2382919893967788
decade_1970s: -1.0757164001020811
decade_2010s: 1.013283546309374
gender_Female: -0.8494663569747299
decade_2000s: 0.8091733045446412
decade_1990s: 0.6979532236261956
decade_1950s: -0.6324400121276443
decade_1980s: -0.388972736826169
gender_Male: 0.380479586630923
decade_2020s: 0.34602429362892495
genre_pop: -0.19745072067859343
genre_rb: -0.10904147676397868


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Genre Coefficients:

genre_rap: The highest positive coefficient (3.08) suggests that being in the rap genre is strongly associated with increasing the probability of a specific topic.
genre_country and genre_rock: These have significant negative coefficients (-1.80 and -1.44, respectively), indicating that these genres are associated with decreasing the probability of that topic.
genre_pop and genre_rb: Smaller negative coefficients suggest a weaker association with decreasing the topic probability.
Decade Coefficients:

Positive coefficients for decade_2010s, decade_2000s, decade_1990s, and decade_2020s (ranging from 0.35 to 1.01) indicate these decades are positively associated with the topic. Particularly, the 2010s show a strong positive association.
Negative coefficients for decade_1960s, decade_1970s, decade_1950s, and decade_1980s (ranging from -0.39 to -1.24) suggest these decades are negatively associated with the topic, with the 1960s showing the strongest negative association.
Gender Coefficients:

gender_Female: A negative coefficient (-0.85) suggests that female artists are negatively associated with the topic.
gender_Male: A positive but smaller coefficient (0.38) suggests a weaker positive association of male artists with the topic.
Interpreting the Coefficients:
Positive Coefficients: Indicate a positive association with the likelihood of the predicted topic. For example, a rap song is more likely to be associated with the topic in question.
Negative Coefficients: Suggest a negative association, meaning those features make the topic less likely. For example, a country song is less likely to be associated with the topic.
Magnitude of Coefficients: The larger the absolute value of a coefficient, the stronger its impact on the model's prediction. For instance, genre_rap has a more substantial impact than gender_Male.

In [ ]:
parameters = model.get_params()
parameters

In [ ]:
coefficients = model.coef_
coefficients

In [ ]:
#extracting the feature weight for gender varaible

model.fit(X_train, y_train)

# Get the feature names after OneHotEncoding
feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out()

# Find the column indices for the 'gender' feature
gender_feature_indices = [i for i, feature in enumerate(feature_names) if 'gender' in feature]

# Extract the coefficients for the 'gender' feature from the model
gender_coefficients = model.coef_[:, gender_feature_indices]

# Print the gender coefficients
print(gender_coefficients)


Improvement on regression model: hyper parameter tuning


In [ ]:
# Print the number of unique classes
print("Number of unique classes:", len(top10_y.unique()))

# Print the count of instances in each class
print("Count of instances in each class:\n", top10_y.value_counts())


In [ ]:
# Extracting the coefficients
coefficients = best_model.coef_
coefficients

# Mapping coefficients to feature names
coef_dict = dict(zip(feature_names, coefficients))

# Extract and analyze coefficients related to 'gender'
gender_coefs = {k: v for k, v in coef_dict.items() if 'gender' in k}
print(gender_coefs)

# Further steps would be to evaluate the model and interpret these coefficients.